In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/crime_rate

/content/drive/MyDrive/crime_rate


In [4]:
#pip install schedule

In [5]:
import pandas as pd
import os
import calendar
import schedule
import threading
import time
import requests

In [ ]:
# #obtain the data from the api.
# # API endpoint URL
# url = "https://data.police.uk/api/crimes-street/all-crime"

# # Parameters
# start_date = "2023-01"
# end_date = "2023-03"
# params = {"date": f"{start_date},{end_date}"}

# # Make the API call
# response = requests.get(url, params=params)
# data = response.json()

# # Convert JSON response to DataFrame
# df = pd.DataFrame(data)



In [74]:
#initially load the dataset and merge folders
input_folder = 'input'
all_dataframes = []
for foldername, subfolders, filenames in os.walk(input_folder):
    for filename in filenames:
        if filename.endswith('.csv'):
            file_path = os.path.join(foldername, filename)
            street_name = os.path.splitext(filename)[0]  # Extract the street name from the file name
            df = pd.read_csv(file_path)
            df["Street"] = street_name  # Add a new column "Street" with the street name
            all_dataframes.append(df)
merged_df = pd.concat(all_dataframes, ignore_index=True)

In [78]:
merged_df.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context,Street
0,0df803f3e708c85f7c0235bed8ed7b49f9ef1a8e4e6685...,2023-01,Cumbria Constabulary,Cumbria Constabulary,-3.382903,54.871255,On or near Fell Close,E01019126,Allerdale 001A,Public order,Local resolution,NaN,2023-01-cumbria-street
1,5a554a4a75242dd25d0c1cde2f0f29c78e5901b71ea525...,2023-01,Cumbria Constabulary,Cumbria Constabulary,-3.381283,54.869907,On or near Skiddaw Close,E01019126,Allerdale 001A,Violence and sexual offences,Unable to prosecute suspect,NaN,2023-01-cumbria-street
2,cfb8b638743691ac24d3de33379a25fcc39ed6a55a6192...,2023-01,Cumbria Constabulary,Cumbria Constabulary,-3.387217,54.868015,On or near Eden Street,E01019126,Allerdale 001A,Violence and sexual offences,Unable to prosecute suspect,NaN,2023-01-cumbria-street
3,92d5c5328da8da696c8ca0b844b3c18454e1173d0c6ab5...,2023-01,Cumbria Constabulary,Cumbria Constabulary,-3.387001,54.871262,On or near Waver Court,E01019126,Allerdale 001A,Violence and sexual offences,Unable to prosecute suspect,NaN,2023-01-cumbria-street
4,b5bbd5db69814b9e95511926ef1028680db6eb2d5eae02...,2023-01,Cumbria Constabulary,Cumbria Constabulary,-3.384917,54.861382,On or near Greenrow Meadows,E01019126,Allerdale 001A,Violence and sexual offences,Action to be taken by another organisation,NaN,2023-01-cumbria-street


In [79]:
# Remove the year and month from the 'Street' column
merged_df['Street'] = merged_df['Street'].str.replace(r'^\d{4}-\d{2}-', '')

<ipython-input-79-7995c0e02f1a>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  merged_df['Street'] = merged_df['Street'].str.replace(r'^\d{4}-\d{2}-', '')


In [80]:
merged_df.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context,Street
0,0df803f3e708c85f7c0235bed8ed7b49f9ef1a8e4e6685...,2023-01,Cumbria Constabulary,Cumbria Constabulary,-3.382903,54.871255,On or near Fell Close,E01019126,Allerdale 001A,Public order,Local resolution,NaN,cumbria-street
1,5a554a4a75242dd25d0c1cde2f0f29c78e5901b71ea525...,2023-01,Cumbria Constabulary,Cumbria Constabulary,-3.381283,54.869907,On or near Skiddaw Close,E01019126,Allerdale 001A,Violence and sexual offences,Unable to prosecute suspect,NaN,cumbria-street
2,cfb8b638743691ac24d3de33379a25fcc39ed6a55a6192...,2023-01,Cumbria Constabulary,Cumbria Constabulary,-3.387217,54.868015,On or near Eden Street,E01019126,Allerdale 001A,Violence and sexual offences,Unable to prosecute suspect,NaN,cumbria-street
3,92d5c5328da8da696c8ca0b844b3c18454e1173d0c6ab5...,2023-01,Cumbria Constabulary,Cumbria Constabulary,-3.387001,54.871262,On or near Waver Court,E01019126,Allerdale 001A,Violence and sexual offences,Unable to prosecute suspect,NaN,cumbria-street
4,b5bbd5db69814b9e95511926ef1028680db6eb2d5eae02...,2023-01,Cumbria Constabulary,Cumbria Constabulary,-3.384917,54.861382,On or near Greenrow Meadows,E01019126,Allerdale 001A,Violence and sexual offences,Action to be taken by another organisation,NaN,cumbria-street


In [81]:
# Remove hyphens from the 'Street' column
merged_df['Street'] = merged_df['Street'].str.replace('-', ' ')

In [82]:
merged_df.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context,Street
0,0df803f3e708c85f7c0235bed8ed7b49f9ef1a8e4e6685...,2023-01,Cumbria Constabulary,Cumbria Constabulary,-3.382903,54.871255,On or near Fell Close,E01019126,Allerdale 001A,Public order,Local resolution,NaN,cumbria street
1,5a554a4a75242dd25d0c1cde2f0f29c78e5901b71ea525...,2023-01,Cumbria Constabulary,Cumbria Constabulary,-3.381283,54.869907,On or near Skiddaw Close,E01019126,Allerdale 001A,Violence and sexual offences,Unable to prosecute suspect,NaN,cumbria street
2,cfb8b638743691ac24d3de33379a25fcc39ed6a55a6192...,2023-01,Cumbria Constabulary,Cumbria Constabulary,-3.387217,54.868015,On or near Eden Street,E01019126,Allerdale 001A,Violence and sexual offences,Unable to prosecute suspect,NaN,cumbria street
3,92d5c5328da8da696c8ca0b844b3c18454e1173d0c6ab5...,2023-01,Cumbria Constabulary,Cumbria Constabulary,-3.387001,54.871262,On or near Waver Court,E01019126,Allerdale 001A,Violence and sexual offences,Unable to prosecute suspect,NaN,cumbria street
4,b5bbd5db69814b9e95511926ef1028680db6eb2d5eae02...,2023-01,Cumbria Constabulary,Cumbria Constabulary,-3.384917,54.861382,On or near Greenrow Meadows,E01019126,Allerdale 001A,Violence and sexual offences,Action to be taken by another organisation,NaN,cumbria street


In [83]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1457621 entries, 0 to 1457620
Data columns (total 13 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Crime ID               1244622 non-null  object 
 1   Month                  1457621 non-null  object 
 2   Reported by            1457621 non-null  object 
 3   Falls within           1457621 non-null  object 
 4   Longitude              1432548 non-null  float64
 5   Latitude               1432548 non-null  float64
 6   Location               1457621 non-null  object 
 7   LSOA code              1394481 non-null  object 
 8   LSOA name              1394481 non-null  object 
 9   Crime type             1457621 non-null  object 
 10  Last outcome category  1217760 non-null  object 
 11  Context                0 non-null        float64
 12  Street                 1457621 non-null  object 
dtypes: float64(3), object(10)
memory usage: 144.6+ MB


In [84]:
# Check for duplicates based on the 'Crime ID' column
duplicate_rows = merged_df.duplicated(subset=['Crime ID'], keep=False)


# Print the duplicate rows
duplicate_rows.value_counts()

False    1195925
True      261696
dtype: int64

In [85]:
#remove duplicates
merged_df = merged_df.drop_duplicates(subset=['Crime ID'], keep='first')

In [86]:
# Check for duplicates based on the 'Crime ID' column
duplicate_rows = merged_df.duplicated(subset=['Crime ID'], keep=False)


# Print the duplicate rows
duplicate_rows.value_counts()

False    1215334
dtype: int64

In [87]:
merged_df.tail()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context,Street
1457616,8ae1bd253db4284594fc96f57a7e3cba0592ca3ddaf3c1...,2023-03,Leicestershire Police,Leicestershire Police,NaN,NaN,No Location,NaN,NaN,Other crime,Under investigation,NaN,leicestershire street
1457617,28ce6e3473f8a24c4efa3018e0115238ab612635e41970...,2023-03,Leicestershire Police,Leicestershire Police,NaN,NaN,No Location,NaN,NaN,Other crime,Action to be taken by another organisation,NaN,leicestershire street
1457618,a5504b64a6d52a4f4cbcdb254f2130b949ec410aa75eb4...,2023-03,Leicestershire Police,Leicestershire Police,NaN,NaN,No Location,NaN,NaN,Other crime,Under investigation,NaN,leicestershire street
1457619,1c070cfb86af48e08a14847110f73e47f61704ccca6f3c...,2023-03,Leicestershire Police,Leicestershire Police,NaN,NaN,No Location,NaN,NaN,Other crime,Under investigation,NaN,leicestershire street
1457620,dbf77012fbba9572d45938ce5c3e1ba61478f80826123e...,2023-03,Leicestershire Police,Leicestershire Police,NaN,NaN,No Location,NaN,NaN,Other crime,Under investigation,NaN,leicestershire street


In [88]:
#check for missing values
merged_df.isnull().sum()

Crime ID                       1
Month                          0
Reported by                    0
Falls within                   0
Longitude                  24027
Latitude                   24027
Location                       0
LSOA code                  33712
LSOA name                  33712
Crime type                     0
Last outcome category       9855
Context                  1215334
Street                         0
dtype: int64

In [89]:
#drop context
merged_df = merged_df.drop('Context', axis=1)

In [90]:
#drop Crime ID as it will be generated when I load data into Snowflake
merged_df = merged_df.drop('Crime ID', axis=1)

In [91]:
#deal with missing long and latitude
# Calculate the mean latitude and longitude values for each unique area in the "Falls Within" column
mean_coordinates = merged_df.groupby('Falls within')['Latitude', 'Longitude'].mean()

# Iterate over the rows with missing latitude and longitude values
for index, row in merged_df[merged_df['Latitude'].isnull() | merged_df['Longitude'].isnull()].iterrows():
    area = row['Falls within']

    # Check if the mean coordinates for the area are available
    if area in mean_coordinates.index:
        mean_lat = mean_coordinates.loc[area, 'Latitude']
        mean_lon = mean_coordinates.loc[area, 'Longitude']

        # Fill in the missing latitude and longitude values with the mean coordinates
        merged_df.at[index, 'Latitude'] = mean_lat
        merged_df.at[index, 'Longitude'] = mean_lon

<ipython-input-91-a5fc5e0e288a>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  mean_coordinates = merged_df.groupby('Falls within')['Latitude', 'Longitude'].mean()


In [92]:
#check for missing values
merged_df.isnull().sum()

Month                        0
Reported by                  0
Falls within                 0
Longitude                    0
Latitude                     0
Location                     0
LSOA code                33712
LSOA name                33712
Crime type                   0
Last outcome category     9855
Street                       0
dtype: int64

In [93]:
# Replace missing values in the "Last Outcome Category" column with "unknown"
merged_df['Last outcome category'] = merged_df['Last outcome category'].fillna('unknown')


In [94]:
#check for missing values
merged_df.isnull().sum()

Month                        0
Reported by                  0
Falls within                 0
Longitude                    0
Latitude                     0
Location                     0
LSOA code                33712
LSOA name                33712
Crime type                   0
Last outcome category        0
Street                       0
dtype: int64

In [95]:
#replace missing values for lsoa code and name with not available
merged_df['LSOA code'] = merged_df['LSOA code'].fillna('Not Available')
merged_df['LSOA name'] = merged_df['LSOA name'].fillna('Not Available')


In [96]:
#check for missing values
merged_df.isnull().sum()

Month                    0
Reported by              0
Falls within             0
Longitude                0
Latitude                 0
Location                 0
LSOA code                0
LSOA name                0
Crime type               0
Last outcome category    0
Street                   0
dtype: int64

In [97]:

# Convert the 'Month' column to datetime format
merged_df['Month'] = pd.to_datetime(merged_df['Month'])

# Extract the year and month into separate columns
merged_df['Year'] = merged_df['Month'].dt.year
merged_df['Month'] = merged_df['Month'].dt.month

In [98]:
merged_df.head()

,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Street,Year
0,1,Cumbria Constabulary,Cumbria Constabulary,-3.382903,54.871255,On or near Fell Close,E01019126,Allerdale 001A,Public order,Local resolution,cumbria street,2023
1,1,Cumbria Constabulary,Cumbria Constabulary,-3.381283,54.869907,On or near Skiddaw Close,E01019126,Allerdale 001A,Violence and sexual offences,Unable to prosecute suspect,cumbria street,2023
2,1,Cumbria Constabulary,Cumbria Constabulary,-3.387217,54.868015,On or near Eden Street,E01019126,Allerdale 001A,Violence and sexual offences,Unable to prosecute suspect,cumbria street,2023
3,1,Cumbria Constabulary,Cumbria Constabulary,-3.387001,54.871262,On or near Waver Court,E01019126,Allerdale 001A,Violence and sexual offences,Unable to prosecute suspect,cumbria street,2023
4,1,Cumbria Constabulary,Cumbria Constabulary,-3.384917,54.861382,On or near Greenrow Meadows,E01019126,Allerdale 001A,Violence and sexual offences,Action to be taken by another organisation,cumbria street,2023


In [99]:
# Create a dictionary mapping month numbers to month names
month_mapping = {
    1: 'January',
    2: 'February',
    3: 'March',
    4: 'April',
    5: 'May',
    6: 'June',
    7: 'July',
    8: 'August',
    9: 'September',
    10: 'October',
    11: 'November',
    12: 'December'
}

# Map the month numbers to month names
merged_df['Month'] = merged_df['Month'].map(month_mapping)

In [100]:
merged_df.head()

,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Street,Year
0,January,Cumbria Constabulary,Cumbria Constabulary,-3.382903,54.871255,On or near Fell Close,E01019126,Allerdale 001A,Public order,Local resolution,cumbria street,2023
1,January,Cumbria Constabulary,Cumbria Constabulary,-3.381283,54.869907,On or near Skiddaw Close,E01019126,Allerdale 001A,Violence and sexual offences,Unable to prosecute suspect,cumbria street,2023
2,January,Cumbria Constabulary,Cumbria Constabulary,-3.387217,54.868015,On or near Eden Street,E01019126,Allerdale 001A,Violence and sexual offences,Unable to prosecute suspect,cumbria street,2023
3,January,Cumbria Constabulary,Cumbria Constabulary,-3.387001,54.871262,On or near Waver Court,E01019126,Allerdale 001A,Violence and sexual offences,Unable to prosecute suspect,cumbria street,2023
4,January,Cumbria Constabulary,Cumbria Constabulary,-3.384917,54.861382,On or near Greenrow Meadows,E01019126,Allerdale 001A,Violence and sexual offences,Action to be taken by another organisation,cumbria street,2023


In [101]:
#monthly crime by counts in the uk
# Create a pivot table to count crime types by street and month
pivot_table = merged_df.pivot_table(index=['Street', 'Year', 'Month'], columns='Crime type', aggfunc='size', fill_value=0)

# Reset the index to flatten the pivot table
pivot_table = pivot_table.reset_index()

pivot_table.head()

Crime type,Street,Year,Month,Anti-social behaviour,Bicycle theft,Burglary,Criminal damage and arson,Drugs,Other crime,Other theft,Possession of weapons,Public order,Robbery,Shoplifting,Theft from the person,Vehicle crime,Violence and sexual offences
0,avon and somerset street,2023,February,0,190,564,1215,157,168,892,71,1384,160,1051,98,795,4448
1,avon and somerset street,2023,January,0,149,602,1045,208,237,869,60,1235,123,991,69,709,4715
2,avon and somerset street,2023,March,0,190,530,1091,206,219,991,71,1481,134,1110,56,885,4869
3,bedfordshire street,2023,February,0,42,196,288,149,83,337,51,323,52,273,38,331,1453
4,bedfordshire street,2023,January,0,52,235,305,158,92,374,49,309,38,202,44,395,1550


In [102]:
#check for missing values
pivot_table.isnull().sum()

Crime type
Street                          0
Year                            0
Month                           0
Anti-social behaviour           0
Bicycle theft                   0
Burglary                        0
Criminal damage and arson       0
Drugs                           0
Other crime                     0
Other theft                     0
Possession of weapons           0
Public order                    0
Robbery                         0
Shoplifting                     0
Theft from the person           0
Vehicle crime                   0
Violence and sexual offences    0
dtype: int64

In [103]:
pivot_table['Street'].unique()

array(['avon and somerset street', 'bedfordshire street',
       'cambridgeshire street', 'cheshire street',
       'city of london street', 'cleveland street', 'cumbria street',
       'derbyshire street', 'dorset street', 'durham street',
       'dyfed powys street', 'essex street', 'gloucestershire street',
       'gwent street', 'hampshire street', 'hertfordshire street',
       'humberside street', 'kent street', 'lancashire street',
       'leicestershire street', 'lincolnshire street',
       'merseyside street', 'metropolitan street', 'norfolk street',
       'north wales street', 'north yorkshire street',
       'northamptonshire street', 'northern ireland street',
       'northumbria street', 'nottinghamshire street',
       'south wales street', 'south yorkshire street',
       'staffordshire street', 'suffolk street', 'surrey street',
       'sussex street', 'thames valley street', 'warwickshire street',
       'west mercia street', 'west midlands street',
       'west york

In [33]:
#pip install snowflake-connector-python

In [104]:
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

In [107]:
print('opening snowflake..')
cnn = snowflake.connector.connect(
    user = 'Boluyeb',
    password = 'Folake10$',
    account = 'in33487.eu-west-2.aws',
    warehouse = 'crime',
    database = 'CRIME_DB',
    schema = 'CRIME_SCHEMA'
)
success, nchunks, nrows, _ = write_pandas(cnn, merged_df, 'STREET_LEVEL_CRIME_DATASET')
print(str(success) +', '+ str(nchunks)+ ', ' +str(nrows))
cnn.close()

opening snowflake..


<ipython-input-107-b9c489782a8f>:10: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.numeric.Int64Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, nchunks, nrows, _ = write_pandas(cnn, merged_df, 'STREET_LEVEL_CRIME_DATASET')


True, 1, 1215334


In [108]:
print('opening snowflake..')
cnn = snowflake.connector.connect(
    user = 'Boluyeb',
    password = 'Folake10$',
    account = 'in33487.eu-west-2.aws',
    warehouse = 'crime',
    database = 'CRIME_DB',
    schema = 'CRIME_SCHEMA'
)
success, nchunks, nrows, _ = write_pandas(cnn, pivot_table, 'MONTHLY_CRIME_COUNTS')
print(str(success) +', '+ str(nchunks)+ ', ' +str(nrows))
cnn.close()

opening snowflake..
True, 1, 125


Regular Dataload at 1 month interval

In [109]:
#load and merge the dataset.
def load_dataframe():
  input_folder = 'input'
  all_dataframes = []
  for foldername, subfolders, filenames in os.walk(input_folder):
      for filename in filenames:
          if filename.endswith('.csv'):
              file_path = os.path.join(foldername, filename)
              street_name = os.path.splitext(filename)[0]  # Extract the street name from the file name
              df = pd.read_csv(file_path)
              df["Street"] = street_name  # Add a new column "Street" with the street name
              all_dataframes.append(df)
  merged_df = pd.concat(all_dataframes, ignore_index=True)
  merged_df['Street'] = merged_df['Street'].str.replace(r'^\d{4}-\d{2}-', '')
  merged_df['Street'] = merged_df['Street'].str.replace('-', ' ')
  return merged_df

In [110]:
#clean the dataset
def cleaner(merged_df):

  merged_df = merged_df.drop_duplicates(subset=['Crime ID'], keep='first') #remove duplicates that exist after merging

  merged_df = merged_df.drop('Context', axis=1) #drop context column

  merged_df = merged_df.drop('Crime ID', axis=1) #drop Crime ID as it will be generated when I load data into Snowflake


  mean_coordinates = merged_df.groupby('Falls within')['Latitude', 'Longitude'].mean() #deal with missing long and latitude. Calculate the mean latitude and longitude values for each unique area in the "Falls Within" column


  for index, row in merged_df[merged_df['Latitude'].isnull() | merged_df['Longitude'].isnull()].iterrows(): # Iterate over the rows with missing latitude and longitude values
      area = row['Falls within']


      if area in mean_coordinates.index:        # Check if the mean coordinates for the area are available
          mean_lat = mean_coordinates.loc[area, 'Latitude']
          mean_lon = mean_coordinates.loc[area, 'Longitude']


          merged_df.at[index, 'Latitude'] = mean_lat # Fill in the missing latitude and longitude values with the mean coordinates
          merged_df.at[index, 'Longitude'] = mean_lon

  merged_df['Last outcome category'] = merged_df['Last outcome category'].fillna('unknown') # Replace missing values in the "Last Outcome Category" column with "unknown"

  merged_df['LSOA code'] = merged_df['LSOA code'].fillna('Not Available') #replace missing values for lsoa code and name with not available
  merged_df['LSOA name'] = merged_df['LSOA name'].fillna('Not Available')


  merged_df['Month'] = pd.to_datetime(merged_df['Month']) # Convert the 'Month' column to datetime format


  merged_df['Year'] = merged_df['Month'].dt.year # Extract the year and month into separate columns
  merged_df['Month'] = merged_df['Month'].dt.month


  month_mapping = { # Create a dictionary mapping month numbers to month names
      1: 'January',
      2: 'February',
      3: 'March',
      4: 'April',
      5: 'May',
      6: 'June',
      7: 'July',
      8: 'August',
      9: 'September',
      10: 'October',
      11: 'November',
      12: 'December'
  }


  merged_df['Month'] = merged_df['Month'].map(month_mapping) # Map the month numbers to month names
  return merged_df



In [111]:
#transform data
def transform(merged_df):

  pivot_table = merged_df.pivot_table(index=['Street', 'Year', 'Month'], columns='Crime type', aggfunc='size', fill_value=0)  #monthly crime by counts in the uk, Create a pivot table to count crime types by street and month

  pivot_table = pivot_table.reset_index() # Reset the index to flatten the pivot table

  return pivot_table




In [112]:
#load data into snowflake
def load(df,table_name):

  print('opening snowflake..') #establish connection
  cnn = snowflake.connector.connect(
      user = 'Boluyeb',
      password = 'Folake10$',
      account = 'in33487.eu-west-2.aws',
      warehouse = 'crime',
      database = 'CRIME_DB',
      schema = 'CRIME_SCHEMA'
  )

  success, nchunks, nrows, _ = write_pandas(cnn, df, table_name)
  print(str(success) +', '+ str(nchunks)+ ', ' +str(nrows))
  cnn.close()

In [117]:
def delete_input_folder():
    folder_path = 'input'
    for foldername, subfolders, filenames in os.walk(folder_path):
        for filename in filenames:
            file_path = os.path.join(foldername, filename)
            os.remove(file_path)  # Delete individual files within the folder

    os.rmdir(folder_path)  # Delete the empty directory



In [116]:
def main():
  df = None

  folder_path = 'input'
  if os.path.exists(folder_path):

    df = load_dataframe() #obtain data

    merged_df = cleaner(df) #clean dataset

    pivot_table = transform(merged_df) #transformed data.

    # pivot_table.to_csv(
    #       "transformed.csv"
    #   ) # TODO: Remove after testing. This makes sure your transform code works

    load(merged_df, 'STREET_LEVEL_CRIME_DATASET') #load the original dataset into the snowflake table

    load(pivot_table, 'MONTHLY_CRIME_COUNTS') #load the pivot table dataset into the snowflake table


    # Call the function to delete the "input" folder
    delete_input_folder()

  else:
    exit()




In [115]:
#main()

<ipython-input-109-9ba7510d038c>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  merged_df['Street'] = merged_df['Street'].str.replace(r'^\d{4}-\d{2}-', '')
<ipython-input-110-e7fe2fa27378>:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  mean_coordinates = merged_df.groupby('Falls within')['Latitude', 'Longitude'].mean() #deal with missing long and latitude. Calculate the mean latitude and longitude values for each unique area in the "Falls Within" column


opening snowflake..


<ipython-input-112-50d2f8c1ac62>:14: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.numeric.Int64Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, nchunks, nrows, _ = write_pandas(cnn, df, table_name)


True, 1, 399037
opening snowflake..
True, 1, 40


In [ ]:
#automated update process
def run_threaded(job_func):
    job_thread = threading.Thread(target=job_func)
    job_thread.start()

schedule.every(30).days.do(run_threaded, main)